In [1]:
import os 
import pandas as pd 
from zero_shot import get_predictions
from datasets import load_dataset
from sklearn.model_selection import train_test_split

/Users/emrekuru/Developer/Fine-Tune/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("openai_api_key.txt") as f:
    api_key = f.read().strip()

os.environ["OPENAI_API_KEY"] = api_key

In [3]:
dataset = load_dataset("financial_phrasebank", "sentences_50agree", trust_remote_code=True)

df = pd.DataFrame(dataset['train'])

# Stratify split into train, validation, and test
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['sentence'], df['label'], test_size=0.2, stratify=df['label'], random_state=42
)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.1, stratify=train_labels, random_state=42
)

train_df = pd.DataFrame({'sentence': train_texts, 'label': train_labels})
val_df = pd.DataFrame({'sentence': val_texts, 'label': val_labels})
test_df = pd.DataFrame({'sentence': test_texts, 'label': test_labels})

In [4]:
predictions = get_predictions(test_df["sentence"].tolist(), "gpt-4o-mini", batch_size=100)

Processing Batches: 100%|██████████| 10/10 [09:41<00:00, 58.20s/it]


In [5]:
pd.DataFrame(predictions, index=test_df.index, columns=["prediction"]).to_csv("predictions/gpt-4.csv", index=True)